In [2]:
from dotenv import load_dotenv
import os
from pydantic import create_model
import inspect, json
from inspect import Parameter
from openai import OpenAI

print(load_dotenv())

os.environ['OPENAI_API_KEY'] = os.getenv("OPEN_AI_API_KEY")

True


**Define the functions**

In [2]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return num1 + num1 - num2 - num2

In [35]:
sbc_json = jsonschema(abc)
sbc_json

{'name': 'abc',
 'description': 'Compute abc between two numbers',
 'parameters': {'properties': {'num1': {'title': 'Num1', 'type': 'integer'},
   'num2': {'title': 'Num2', 'type': 'integer'}},
  'required': ['num1', 'num2'],
  'title': 'Input for `abc`',
  'type': 'object'}}

In [16]:
model_name = "gpt-3.5-turbo"

**Ask GPT**

In [17]:
client = OpenAI()

response = client.chat.completions.create(
  model= model_name,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me a joke"},
  ],
)

In [18]:
response.choices[0].message.content

"Sure! Here you go: Why don't scientists trust atoms? Because they make up everything!"

In [19]:
abc(1, 5)

-8

In [23]:
messages= [
    {"role": "user", "content": "Compute abc between 1 and 5"}
]

# Pass th function to GPT model
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    functions=[sbc_json],
    function_call="auto",
    temperature=0
)

**Executing the function by extracting the info from the output of the model**

In [30]:
print(response.choices[0].message.function_call)
print(response.choices[0].message.function_call.arguments)
print(type(response.choices[0].message.function_call.arguments))

FunctionCall(arguments='{"num1":1,"num2":5}', name='abc')
{"num1":1,"num2":5}
<class 'str'>


In [33]:
func_name = response.choices[0].message.function_call.name
func_args = json.loads(response.choices[0].message.function_call.arguments)
print("Function name:", func_name)
print("Function arguments:", func_args)
print(type(func_args))

Function name: abc
Function arguments: {'num1': 1, 'num2': 5}
<class 'dict'>


In [34]:
if func_name == 'abc':
    result = abc(**func_args)
print(result)

-8
